# Digit recognizer

In [1]:
#Configuraciones iniciales
options(scipen = 999, repr.plot.width=4, repr.plot.height= 4, warn = -1)

In [143]:
#Instalación y carga de paquetes necesarios
list.of.packages <- c('fields', 'ggmap', 'gridExtra', 'lbfgsb3')

new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, repos = "https://cran.r-project.org")

library(fields)
library(ggmap)
library(gridExtra)
library(lbfgsb3)

In [144]:
df.train <- read.csv("train.csv")

In [145]:
X <- as.matrix(df.train[, 2:ncol(df.train)])
y <- df.train$label
y[y == 0] <- 10

X <- X / 255#Scale values
dim(X)

[1] 42000   784

In [146]:
#Creamos la función sigmoidal
sigmoid <- function(z) {
  
  g <- 1 / (1 + exp(1) ^ (-z))
}

#Función para calcular el gradiente de la función sigmoidal
sigmoidGradient <- function(z) {
    
    z <- sigmoid(z)
    g  <- z * (1 - z)
    g
}

In [147]:
#Funcion para calcular el costo (con regularización de parámetros)

nnCostFunction <- function(nn_params) {
    
    Theta1 <- matrix(nn_params[1:(hidden_layer_size *(input_layer_size + 1))], hidden_layer_size, input_layer_size +1)
    Theta2 <- matrix(nn_params[(1 + hidden_layer_size * (input_layer_size + 1)):length(nn_params)], 
                     num_labels, hidden_layer_size +1)
    
    Theta1_nz <- Theta1[, 2:ncol(Theta1)]
    Theta2_nz <- Theta2[, 2:ncol(Theta2)]
    
    m <- nrow(X)
    J <- 0
    
    X <- cbind(1, X)
    a1 <- X
    z2 <- a1 %*% t(Theta1)
    a2 <- sigmoid(z2)
    a2 <- cbind(1, a2)
    z3 <- a2 %*% t(Theta2)
    a3 <- sigmoid(z3)
    
    hyp <- a3
    y_mat <- diag(num_labels)
    y_mat <- y_mat[, y]
    
    inner_value <- - t(y_mat) * log(hyp) - (1 - t(y_mat)) * log(1 - hyp)
    J_noreg <- (1/m) * sum(sum(inner_value))
    reg_term <- (lambda / (2 * m)) * sum(sum(sum(Theta1_nz ^ 2)) + sum(sum(Theta2_nz ^ 2)))
    
    J <- J_noreg + reg_term
    
    J
    
}

In [148]:
#Función para calcular los gradientes (con regularización de parámetros)
nnGradFunction <- function(nn_params) {
    
    Theta1 <- matrix(nn_params[1:(hidden_layer_size *(input_layer_size + 1))], hidden_layer_size, input_layer_size +1)
    Theta2 <- matrix(nn_params[(1 + hidden_layer_size * (input_layer_size + 1)):length(nn_params)], 
                     num_labels, hidden_layer_size +1)
    
    Theta1_grad <- matrix(0, ncol = ncol(Theta1), nrow = nrow(Theta1))
    Theta2_grad <- matrix(0, ncol = ncol(Theta1), nrow = nrow(Theta2))
    
    Theta1_nz <- Theta1[, 2:ncol(Theta1)]
    Theta2_nz <- Theta2[, 2:ncol(Theta2)]
    
    m <- nrow(X)
   
    X <- cbind(1, X)
    a1 <- X
    z2 <- a1 %*% t(Theta1)
    a2 <- sigmoid(z2)
    a2 <- cbind(1, a2)
    z3 <- a2 %*% t(Theta2)
    a3 <- sigmoid(z3)
    
    hyp <- a3
    y_mat <- diag(num_labels)
    y_mat <- y_mat[, y]
    
    delta3 <- a3 - t(y_mat)
    z2 <- cbind(1, z2)
    delta2 <- (delta3 %*% Theta2) * sigmoidGradient(z2)
    delta2 <- delta2[, 2:ncol(delta2)]
    
    cap_delta1 <- 0
    cap_delta2 <- 0
    
    cap_delta1 <- cap_delta1 + t(delta2) %*% a1
    cap_delta2 <- cap_delta2 + t(delta3) %*% a2
    
    Theta1[, 1] <- 0
    Theta2[, 1] <- 0
    
    Theta1_grad <- (1 / m) * (cap_delta1 + lambda * Theta1)
    Theta2_grad <- (1 / m) * (cap_delta2 + lambda * Theta2)
    
    grad <- c(c(Theta1_grad), c(Theta2_grad))
}

In [149]:
#Entrenamiento de la red neuronal----------------------------------------------------

#Inicializamos los parámetros de manera aleatoria
randInitializeWeights <- function(L_in, L_out) {
    
    W <- matrix(0, L_out, 1 + L_in)
    epsilon_init <- 0.12
  
    rnd <- runif(L_out * (1 + L_in))
    rnd <- matrix(rnd,L_out,1 + L_in)
    W <- rnd * 2 * epsilon_init - epsilon_init
    W
}

n <- ncol(X)
m <- nrow(X)

input_layer_size <- n
hidden_layer_size <- 25
num_labels <- 10
lambda <- 1

#initial_Theta1 <- randInitializeWeights(input_layer_size, hidden_layer_size)
#initial_Theta2 <- randInitializeWeights(hidden_layer_size, num_labels)

initial_Theta1 <- matrix(runif((n+1)*hidden_layer_size), nrow = hidden_layer_size, ncol=n + 1)
initial_Theta2 <- matrix(runif(num_labels * (hidden_layer_size + 1)), nrow = num_labels, ncol = hidden_layer_size + 1)

initial_nn_params <- c(c(initial_Theta1), c(initial_Theta2))

In [ ]:
results <- optim(initial_nn_params, fn = nnCostFunction, gr = nnGradFunction, method = "BFGS", 
               control = list(maxit=1000, trace=1, REPORT=1) )

theta <- results$par
J <- results$value

cat("Costo final:", J)

In [108]:
Theta1 <- matrix(theta[1:(hidden_layer_size *(input_layer_size + 1))], hidden_layer_size, input_layer_size +1)
Theta2 <- matrix(theta[(1 + hidden_layer_size * (input_layer_size + 1)):length(nn_params)], 
                     num_labels, hidden_layer_size +1)

In [109]:
#Calculamos la precisión de la red

predict <- function(Theta1, Theta2, X) {
    
    m <- nrow(X)
    num_labels <- nrow(Theta2)
    
    p <- rep(0, m)
    
    h1 <- sigmoid(cbind(1, X) %*% t(Theta1))
    h2 <- sigmoid(cbind(1, h1) %*% t(Theta2))
    
    p <- apply(h2, 1, which.max)
    p
}

In [129]:
df.test <- read.csv("test.csv")
X <- as.matrix(df.test[, 2:ncol(df.test)])
X <- cbind(0, X)
X <- X / 255

In [130]:
#el test comienza en pixel1 a pixel783
#el train comienza en pixel0 a pixel782

In [133]:
table(y)

y
   1    2    3    4    5    6    7    8    9   10 
4684 4177 4351 4072 3795 4137 4401 4063 4188 4132 

In [136]:
predictions <- predict(Theta1, Theta2, X)

table(predictions)
predictions[predictions == 10] <- 0

m <- nrow(df.test)
ids <- 1:m

predictions
   1    2    3    4    5    6    7    8    9   10 
3342 5452  834 3764  183  644 4699 4291  322 4469 

In [137]:
table(predictions)

predictions
   0    1    2    3    4    5    6    7    8    9 
4469 3342 5452  834 3764  183  644 4699 4291  322 

In [138]:
submit <- data.frame(ImageId = ids, Label = predictions)

In [139]:
table(predictions)

predictions
   0    1    2    3    4    5    6    7    8    9 
4469 3342 5452  834 3764  183  644 4699 4291  322 

In [140]:
head(submit)

ImageId,Label
1,2
2,0
3,4
4,7
5,8
6,7


In [141]:
write.csv(submit, file = "segundosubmit.csv", row.names = FALSE, quote = FALSE)

In [ ]:
Sys.time()